In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 261 kB in 2s (171 kB/s)
Reading package lis

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "http://challengmcg.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True)

In [ ]:
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [ ]:
relevant_vines = df.filter("total_votes>=20")
relevant_vines.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [ ]:
helpful_vines = relevant_vines.filter("(helpful_votes/total_votes)>=.5")
helpful_vines.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [ ]:
paid_vines = helpful_vines.filter("vine == 'Y'")
paid_vines.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [ ]:
unpaid_vines = helpful_vines.filter("vine == 'N'")
unpaid_vines.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [134]:
# Import function
from pyspark.sql.functions import count
# Total Paid Vine reviews
total_paid = paid_vines.select(count("review_id")).withColumnRenamed('count(review_id)', 'Total Number of Paid Reviews')
total_paid.show()

+----------------------------+
|Total Number of Paid Reviews|
+----------------------------+
|                          94|
+----------------------------+



In [135]:
# Total paid 5 star reviews
from pyspark.sql.types import IntegerType
total_5stars = paid_vines.filter("star_rating == '5'").select(count("star_rating")).withColumnRenamed('count(star_rating)', '5-star Reviews')
total_5stars = total_5stars.withColumn('5-star Reviews', total_5stars['5-star Reviews'].cast(IntegerType()))
total_5stars.show()

+--------------+
|5-star Reviews|
+--------------+
|            48|
+--------------+



In [136]:
paid_5stars = paid_vines.filter("star_rating == '5'").select(count("star_rating")).withColumnRenamed('count(star_rating)', '5-star Paid Reviews')
paid_5stars = paid_5stars.withColumn('5-star Paid Reviews', paid_5stars['5-star Paid Reviews'].cast(IntegerType()))

In [138]:
import pyspark.sql.functions as F
summary_vine = total_paid.join(total_5stars)
summary_vine = summary_vine.join(paid_5stars).withColumn('5-star Paid Reviews %', (F.col("5-star Paid Reviews") / F.col("Total Number of Paid Reviews"))*100).drop("5-star Paid Reviews")

summary_vine.show()

+----------------------------+--------------+---------------------+
|Total Number of Paid Reviews|5-star Reviews|5-star Paid Reviews %|
+----------------------------+--------------+---------------------+
|                          94|            48|    51.06382978723404|
+----------------------------+--------------+---------------------+



In [145]:
# Total Unpaid Vine reviews
total_unpaid = unpaid_vines.select(count("review_id")).withColumnRenamed('count(review_id)', 'Total Number of Unpaid Reviews')
total_unpaid.show()

+------------------------------+
|Total Number of Unpaid Reviews|
+------------------------------+
|                         40471|
+------------------------------+



In [152]:
# Unpaid 5 stars
unpaid_stars = unpaid_vines.filter("star_rating == '5'").select(count("star_rating")).withColumnRenamed('count(star_rating)', '5-star Unpaid Reviews')
unpaid_stars = unpaid_stars.withColumn('5-star Unpaid Reviews', unpaid_stars['5-star Unpaid Reviews'].cast(IntegerType()))
unpaid_stars.show()

+---------------------+
|5-star Unpaid Reviews|
+---------------------+
|                15663|
+---------------------+



In [155]:
unpaid_5stars = unpaid_vines.filter("star_rating == '5'").select(count("star_rating")).withColumnRenamed('count(star_rating)', 'Unpaid')
unpaid_5stars = unpaid_5stars.withColumn('Unpaid', unpaid_5stars['Unpaid'].cast(IntegerType()))

In [157]:
summary_unpaid = total_unpaid.join(unpaid_stars)
summary_unpaid = summary_unpaid.join(unpaid_5stars).withColumn('5-star Unpaid Reviews %', (F.col('Unpaid') / F.col('Total Number of Unpaid Reviews'))*100).drop("Unpaid")

summary_unpaid.show()

+------------------------------+---------------------+-----------------------+
|Total Number of Unpaid Reviews|5-star Unpaid Reviews|5-star Unpaid Reviews %|
+------------------------------+---------------------+-----------------------+
|                         40471|                15663|     38.701786464381904|
+------------------------------+---------------------+-----------------------+

